# Details
I just tried to clip the linear regression prediction -ve values to 0 to submit.
my score got improved from 0.73 to 0.72 

In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm_notebook
from sklearn.preprocessing import LabelEncoder
import gc

DATA_FOLDER = '..//data//'
d_parser = lambda x: pd.datetime.strptime(x,'%Y-%m-%d')

import warnings
warnings.filterwarnings("ignore")

In [2]:
df_lr = pd.read_csv('lr_submission_21jun2020_1914_41_1942_69.csv')
df_lgb= pd.read_csv('sub_lgb_2020-06-27.csv')

In [5]:
df_lr.tail()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
60975,FOODS_3_823_WI_3_evaluation,0.331871,0.176466,0.289034,0.197223,0.925745,0.992402,0.746081,0.433655,0.418415,...,0.689607,0.705773,0.324547,0.742764,0.168596,0.516317,0.796283,0.688678,0.791547,0.955618
60976,FOODS_3_824_WI_3_evaluation,0.115438,0.036724,0.080565,0.460385,0.429863,0.533555,0.635585,0.204662,0.320978,...,-0.082727,0.195501,-0.003956,-0.110246,-0.000157,-0.016642,-0.038098,0.297830,0.429487,0.320374
60977,FOODS_3_825_WI_3_evaluation,0.641413,0.408474,0.145014,0.068307,0.493070,0.680199,1.028450,0.887840,0.857799,...,0.775682,1.120082,0.570969,1.126443,1.081710,0.467448,0.515525,0.665072,0.957232,0.876007
60978,FOODS_3_826_WI_3_evaluation,1.216433,0.864453,1.133411,1.043333,1.304254,1.310452,1.387487,0.607470,1.016896,...,0.926397,1.000704,0.792089,0.785153,1.446784,2.318189,1.384213,1.316972,1.589914,1.656442
60979,FOODS_3_827_WI_3_evaluation,0.812200,0.483913,0.013576,-0.214277,-0.077992,0.353293,0.679045,0.166250,0.120233,...,0.416762,0.777158,0.916590,0.285821,1.185245,1.356139,0.598422,1.317524,1.780964,2.537765


In [6]:
df_lgb.tail()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
60975,FOODS_3_823_WI_3_evaluation,0.672382,0.514574,0.506975,0.498441,0.864101,0.921852,0.858849,0.795645,0.717670,...,0.832486,0.786096,0.679810,0.922110,0.690508,0.785737,0.904249,0.885984,0.853636,0.897197
60976,FOODS_3_824_WI_3_evaluation,0.404458,0.357391,0.362630,0.564612,0.514079,0.440842,0.515030,0.466732,0.426144,...,0.286257,0.295654,0.293121,0.327350,0.331321,0.322235,0.289997,0.439617,0.451773,0.454748
60977,FOODS_3_825_WI_3_evaluation,0.967097,0.826338,0.614687,0.685640,0.821056,0.783448,1.015651,1.180031,0.948468,...,0.929932,1.032057,0.872989,1.234810,1.152285,0.866801,0.914228,0.835887,0.935143,0.905109
60978,FOODS_3_826_WI_3_evaluation,1.173168,1.056398,1.146596,1.298840,1.270049,1.225589,1.252785,1.074721,1.100314,...,1.184882,1.030320,1.044416,1.039907,1.134112,1.691074,1.202416,1.169267,1.358671,1.509651
60979,FOODS_3_827_WI_3_evaluation,0.932972,0.892382,0.696387,0.514644,0.501177,0.547069,0.822885,1.033820,1.153147,...,0.884828,1.240950,1.391004,1.066030,1.638690,1.844011,1.225891,1.739253,1.972744,2.409065


In [14]:
df_lr_clip = pd.DataFrame(np.c_[df_lr['id'], df_lr.drop(['id'], axis=1).clip(lower=0)])

In [15]:
df_lr_clip.head()

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,HOBBIES_1_001_CA_1_validation,1.00556,0.648623,0.650656,0.518594,0.865805,0.951055,1.1604,1.64293,0.990957,...,0.817684,1.63638,1.89837,1.08545,0.767535,0.603109,0.842766,1.39329,1.90521,0.982032
1,HOBBIES_1_002_CA_1_validation,0.142918,0.0130358,0.250872,0,0.0762838,0.143169,0.187158,0.0451036,0.0345343,...,0,0,0.259183,0.45762,0.481458,0.559789,0.289199,0.349793,0.451131,0.34996
2,HOBBIES_1_003_CA_1_validation,0.634521,0.385809,0.351101,0.594982,0.709757,0.554483,0.879784,0.530679,0.401945,...,0.772404,0.950289,0.717766,0.644346,0.439576,0.353961,0.393378,0.879909,1.31687,0.871104
3,HOBBIES_1_004_CA_1_validation,2.28456,1.19433,1.12862,1.49957,2.27797,2.93334,2.16943,2.96672,2.41286,...,1.94912,1.94873,2.21591,1.4181,1.29585,1.31861,1.00227,1.78871,1.7653,1.95191
4,HOBBIES_1_005_CA_1_validation,1.76253,1.12056,1.22372,1.49158,1.9108,1.84736,1.35851,1.40052,1.19314,...,1.90541,1.53125,1.30549,0.870765,1.24251,1.198,0.979159,0.75956,1.24684,1.47138


In [19]:
df_lr.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,1.005558,0.648623,0.650656,0.518594,0.865805,0.951055,1.160397,1.642932,0.990957,...,0.817684,1.636377,1.898372,1.085449,0.767535,0.603109,0.842766,1.393291,1.905214,0.982032
1,HOBBIES_1_002_CA_1_validation,0.142918,0.013036,0.250872,-0.006007,0.076284,0.143169,0.187158,0.045104,0.034534,...,-0.009619,-0.002061,0.259183,0.457620,0.481458,0.559789,0.289199,0.349793,0.451131,0.349960
2,HOBBIES_1_003_CA_1_validation,0.634521,0.385809,0.351101,0.594982,0.709757,0.554483,0.879784,0.530679,0.401945,...,0.772404,0.950289,0.717766,0.644346,0.439576,0.353961,0.393378,0.879909,1.316870,0.871104
3,HOBBIES_1_004_CA_1_validation,2.284565,1.194328,1.128618,1.499571,2.277972,2.933338,2.169426,2.966723,2.412862,...,1.949123,1.948735,2.215911,1.418101,1.295854,1.318608,1.002266,1.788706,1.765296,1.951905
4,HOBBIES_1_005_CA_1_validation,1.762528,1.120563,1.223724,1.491585,1.910796,1.847364,1.358512,1.400519,1.193139,...,1.905410,1.531252,1.305493,0.870765,1.242514,1.198001,0.979159,0.759560,1.246841,1.471377


In [21]:
df_lr_clip.to_csv('lr_clip.csv',index=False)

In [18]:
df_lr_clip.columns = df_lr.columns